In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
path = "../Data/case1.xlsx"

team_df = pd.read_excel(path, sheet_name="Team")
DistanceTeamTeam = pd.read_excel(path, sheet_name="DistanceTeamTeam")
SiteTeam = pd.read_excel(path, sheet_name="SiteTeam")
DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
muST = pd.read_excel(path, sheet_name="muST")
Site = pd.read_excel(path, sheet_name="Site")



In [3]:
Site_team_1 = SiteTeam.iloc[:, 1:]

In [4]:
SiteTeamMatrix = Site_team_1.values
SiteTeamMatrix = np.array(SiteTeamMatrix)
print(SiteTeamMatrix.shape)

(944, 19)


In [5]:
Site

,Site number,Assigned to team number,X coordinate of site,Y coordinate of site,Lambda
0,1,1,40396.198411,75031.802209,52.420262
1,2,1,39815.627403,76367.088611,40.712374
2,3,1,42468.282157,74589.342975,35.826941
3,4,1,49448.556898,83244.108791,99.675205
4,5,1,46549.083899,72970.364429,39.918137
...,...,...,...,...,...
939,940,19,59282.199162,8816.066762,46.985018
940,941,19,56986.705790,4249.778488,67.712368
941,942,19,54013.167704,14977.740090,36.103500
942,943,19,55645.265908,10993.011778,16.150037


FIX LATER THE LAMBDA. DATASET ONLY CONTAINS SUBSET OF ALL LAMBDAS


In [6]:
site_number = Site["Site number"]
team_assigned = Site["Assigned to team number"]
all_lambdas = Site["Lambda"]

In [7]:
all_lambdas

0      52.420262
1      40.712374
2      35.826941
3      99.675205
4      39.918137
         ...    
939    46.985018
940    67.712368
941    36.103500
942    16.150037
943    45.689645
Name: Lambda, Length: 944, dtype: float64

find expected rewards: 
$$
\mathbb{E}_{\omega \in \Omega} (a_{ij}^\omega)
$$

In [8]:
#Planners expected rewards
alpha, beta, gamma, delta = 1.451, 0.138, -0.287, 0.057
#expected rewards same size as SiteTeamMatrix
expected_rewards = np.empty((19,944)) * 0.0

n_teams = 19
n_sites = 944
k = 5
team_size = 2


for i in range(n_teams):
    for j in range(n_sites):
        expected_rewards[i][j] = all_lambdas[j] * (alpha + beta * (np.sqrt(12 * k) + np.sqrt(12 * (k - 1))) + gamma * np.sqrt(12) + delta * team_size)


In [9]:
expected_rewards_df = pd.DataFrame(expected_rewards)
DistanceTeamTeam.index = [i for i in range(1, 20)]
DistanceTeamTeam.drop(columns=['Euclidian distance in km between team bases (given the team base coordinates in sheet "Team")'], inplace=True)

Swap depending on location

In [10]:
#Find teams which are closest to each other

def find_closest_teams(DistanceTeamTeam):
    closest_teams = []
    for i in range(19):
        for j in range(19):
            if i != j:
                closest_teams.append((i, j, DistanceTeamTeam.iloc[i, j]))
    closest_teams = sorted(closest_teams, key=lambda x: x[2])
    return closest_teams

closest_teams = find_closest_teams(DistanceTeamTeam)

In [11]:
set_of_teams = [i for i in range(1, 20)]
closest_pairs = []
temp_df = DistanceTeamTeam.copy()


for i in set_of_teams:
    shortest_distance = float('inf')
    for j in set_of_teams: 
        if i != j:
            distance = temp_df.loc[j, i]
            if distance <= shortest_distance:
                shortest_distance = distance
                closest_team = j
    closest_pairs.append((i, closest_team))
    set_of_teams.remove(closest_team)
    set_of_teams.remove(i)


In [12]:
closest_pairs

[(1, 3), (4, 8), (6, 12), (9, 5), (13, 2), (16, 19), (18, 11)]

In [13]:
totalPayOff = SiteTeamMatrix.T * expected_rewards
print(sum(totalPayOff[0]))

4229.623062836057


In [14]:
feasible_locations = DistanceSiteTeam.copy()
feasible_locations.drop(columns=['Manhatten distance in km between sites and teams (given the team base coordinates in sheet "Team")'], inplace=True)
feasible_locations.index = [i for i in range(1, 945)]
feasible_locations = (feasible_locations <= 60).astype(int)


feasible_payoff = feasible_locations.T * expected_rewards


In [15]:
def swaps(pair, siteTeam):
    team1, team2 = pair
    site = siteTeam[team1]
    site = site[site == 1]
    idx = site.index 
    copy_site = siteTeam.copy()
    for i in idx:
        if feasible_payoff[team2][i] > feasible_payoff[team1][i]:
            copy_site[i][team1] = 0
            copy_site[i][team2] = 1
    return copy_site 

In [16]:
def test_optimiser():
    try:
        m = gp.Model("mip1")
        
        x = m.addVar(vtype = GRB.INTEGER, name="x")
        y = m.addVar(vtype = GRB.INTEGER, name="y")
        
        m.setObjective(x + y, GRB.MAXIMIZE)
        
        m.addConstr(2* x +  y <= 4, "c0")
        
        m.optimize()
        
        for v in m.getVars():
            print('%s %g' % (v.varName, v.x))
        
        print('Obj: %g' % m.objVal)
        
    
    
    except gp.GurobiError as e:
        print('Error code ' + str(e.errno) + ": " + str(e))
    except AttributeError:
        print('Encountered an attribute error')


In [17]:
test_optimiser()


Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xdfb0d4bf
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 4 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%
x -0
y 4
Obj: 4


In [18]:
def solve(payoffs: dict, max_tasks: dict, initial_solution: dict, num_teams = int, num_sites = int):
    with gp.Env() as env, gp.Model(env=env) as model:
        m = gp.Model("IC")

        
        
        N =  [i for i in range(1, num_teams + 1)]
        J = [j for j in range(1, num_sites + 1)]
        
        x = {}
        a = payoffs
        
        
        for i in N:
            for j in J:
                    x[i, j] = m.addVar(vtype=GRB.BINARY, name="x_%s_%s" % (i, j))
        
                    
        objective = gp.quicksum(a[i, j] * x[i, j] for i in N for j in J)
        m.setObjective(objective, GRB.MAXIMIZE)
        

        
        for j in J:
            m.addConstr(gp.quicksum(x[i, j] for i in N) <= 1, name=f"c_{j}")
            
        for i in N:
            m.addConstr(gp.quicksum(x[i, j] for j in J) <= max_tasks[i-1], name=f"c_{i}")
            m.addConstr(x[i, j] * a[i,j] >= initial_solution[i, j] * a[i,j], name=f"c_{i}_{j}")    
            

        
        m.optimize()

                
        if m.status == GRB.OPTIMAL:
            m.getAttr('x', x)   
            print('Obj: %g' % m.objVal)
            

        return m.objVal
    
    
            
def expected_payoff(all_lambdas, n_teams, n_sites, team_sizes):
    alpha, beta, gamma, delta = 1.451, 0.138, -0.287, 0.057
    #Same Shape as SiteTeamMatrix
    expected_rewards = np.empty((n_teams, n_sites)) * 0.0


    k = 5



    for i in range(n_teams):
        team_size = team_sizes[i]
        for j in range(n_sites):
            # FIX THIS: 
            # Furthermore, STsji denotes the estimated starting time (in hours) of team i at site sj, which Van Rijn et al. (2020) estimate as 7 AM plus the driving time
            expected_rewards[i][j] = all_lambdas[j] * (alpha + beta * (np.sqrt(12 * k) + np.sqrt(12 * (k - 1))) + gamma * np.sqrt(12) + delta * team_size)
    
    return expected_rewards


    
    
def inititial_sol(SiteTeam: pd.DataFrame) -> dict:
    initial_solution = {}
    shape_params = SiteTeam.shape  
    
    for i in range(1, shape_params[0] + 1):
        for j in range(1, shape_params[1] + 1):
            initial_solution[i, j] = SiteTeam.iloc[i-1, j-1]
    initial_solution1 = {(j, i): initial_solution[i, j] for i in range(1, n_sites + 1) for j in range(1, n_teams + 1)}

    return initial_solution1


    

In [ ]:
init_obj = {}
extra_constraint_obj = {}


initial_solutions = {}
for i in range(1, 101):
    print(f"Solving case {i}")
    path = f"../Generated cases/case{i}.xlsx"

    Team = pd.read_excel(path, sheet_name="Team")
    SiteTeam = pd.read_excel(path, sheet_name="SiteTeam")
    Site = pd.read_excel(path, sheet_name="Site")
    DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
    DistanceSiteTeam = DistanceSiteTeam.drop("Unnamed: 0", axis=1)
    feasible_locations = (DistanceSiteTeam <= 60).astype(int)
    
    info = pd.read_excel(path, sheet_name="Info")
    n_teams, n_sites = info.values[0]

    print(f"Number of teams: {n_teams}, Number of sites: {n_sites}")
    team_sizes = Team.iloc[:, 1]
    all_lambdas = Site.iloc[:, 4]
    expected_rewards = expected_payoff(all_lambdas, n_teams, n_sites, team_sizes)
    expected_rewards = feasible_locations.T.values * expected_rewards

    initial_solution = inititial_sol(SiteTeam)
    
    
    shape_params = expected_rewards.shape

    payoff = {(i, j): expected_rewards[i-1 , j-1] for i in range(1, shape_params[0] + 1) for j in range(1, shape_params[1] + 1)}
    max_tasks = [100 for i in range(n_teams)]

    """"
    SOLVING THE OPTIMISATION PROBLEM
    """

    init_obj[i] = solve(payoff, max_tasks, initial_solution, shape_params[0], shape_params[1])
    

In [20]:
obj

{1: 105092.14017729397,
 2: 172179.49336061341,
 3: 48539.930070980015,
 4: 107378.70969690374,
 5: 101636.4937281354,
 6: 97677.01656482382,
 7: 65501.86490421673,
 8: 62740.684876665655,
 9: 147019.1484098096,
 10: 65087.406837490664,
 11: 109559.46608841803,
 12: 52652.52474904854,
 13: 92012.2685156874,
 14: 58701.925757674224,
 15: 87375.48755922343,
 16: 41281.98247772941,
 17: 71724.63635302543,
 18: 76254.3389687699,
 19: 153765.27813073396,
 20: 115828.6753411458,
 21: 83606.64691312816,
 22: 76145.24005824416,
 23: 78287.98432735472,
 24: 31508.28107253251,
 25: 58122.353811452755,
 26: 44708.34189727258,
 27: 130860.14501687682,
 28: 40942.2602945651,
 29: 27929.481457846465,
 30: 141189.74631493486,
 31: 74301.99679040741,
 32: 104928.19608578013,
 33: 65327.296400471314,
 34: 85053.77281896924,
 35: 168377.43188151123,
 36: 55186.010483472106,
 37: 39863.69233160793,
 38: 138034.06780916892,
 39: 132619.34887059833,
 40: 70377.08746002505,
 41: 141174.93917756152,
 42: 918

In [33]:
path = "../Generated cases/case1.xlsx"

SiteTeam = pd.read_excel(path, sheet_name="SiteTeam")
Site = pd.read_excel(path, sheet_name="Site")
DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
info = pd.read_excel(path, sheet_name="Info")

In [34]:
DistanceSiteTeam = DistanceSiteTeam.drop("Unnamed: 0", axis=1)

In [35]:
feasable_locations = (DistanceSiteTeam <= 60).astype(int)

In [36]:
feasable_locations = feasable_locations.values

In [42]:
n_teams, n_sites = info.values[0]

expected_rewards = expected_payoff(all_lambdas, n_teams, n_sites)

In [43]:
expected_rewards = expected_rewards * feasable_locations.T

In [44]:
payoff = {(i, j): expected_rewards[i-1 , j-1] for i in range(1, 20) for j in range(1, 945)}
max_tasks = [50 for i in range(19)]

In [99]:
solve(payoff, max_tasks, initial_solution, 19, 944)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 982 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0x19b04082
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 99001.857574
Presolve removed 460 rows and 16481 columns
Presolve time: 0.01s
Presolved: 522 rows, 1455 columns, 2586 nonzeros
Found heuristic solution: objective 103117.36798
Variable types: 0 continuous, 1455 integer (1455 binary)
Found heuristic solution: objective 103900.86742

Root relaxation: objective 1.050921e+05, 174 iterations, 0.00 seconds (0.00 work units)

    N

105092.14017729397

In [19]:
path = f"../Generated cases/case{1}.xlsx"

Team = pd.read_excel(path, sheet_name="Team")
SiteTeam = pd.read_excel(path, sheet_name="SiteTeam")
Site = pd.read_excel(path, sheet_name="Site")
DistanceSiteTeam = pd.read_excel(path, sheet_name="DistanceSiteTeam")
DistanceSiteTeam = DistanceSiteTeam.drop("Unnamed: 0", axis=1)
feasible_locations = (DistanceSiteTeam <= 60).astype(int)

info = pd.read_excel(path, sheet_name="Info")
n_teams, n_sites = info.values[0]

print(f"Number of teams: {n_teams}, Number of sites: {n_sites}")
team_sizes = Team.iloc[:, 1]
all_lambdas = Site.iloc[:, 4]
expected_rewards = expected_payoff(all_lambdas, n_teams, n_sites, team_sizes)
expected_rewards = feasible_locations.T.values * expected_rewards

initial_solution = inititial_sol(SiteTeam)


shape_params = expected_rewards.shape

payoff = {(i, j): expected_rewards[i-1 , j-1] for i in range(1, shape_params[0] + 1) for j in range(1, shape_params[1] + 1)}
max_tasks = [100 for i in range(n_teams)]

""""
SOLVING THE OPTIMISATION PROBLEM
"""

solve(payoff, max_tasks, initial_solution, shape_params[0], shape_params[1])


Number of teams: 19, Number of sites: 944
Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-31
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.2.0 23C64)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 982 rows, 17936 columns and 35873 nonzeros
Model fingerprint: 0xe4420720
Variable types: 0 continuous, 17936 integer (17936 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [5e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 107847.51986
Presolve removed 918 rows and 17693 columns
Presolve time: 0.01s
Presolved: 64 rows, 243 columns, 367 nonzeros
Found heuristic solution: objective 108205.27666
Variable types: 0 continuous, 243 integer (243 binary)

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Explored 1 nodes (0 simplex iterati

108205.27666406451

In [20]:
SiteTeam.drop("Unnamed: 0", axis=1, inplace=True)

In [21]:
SiteTeam.T

,0,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
pd.DataFrame(expected_rewards)

,0,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
0,136.074522,105.682739,93.000945,258.740714,103.621028,263.242359,302.823868,140.643303,72.444582,225.701586,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
2,136.074522,105.682739,93.000945,0.000000,0.000000,0.000000,302.823868,140.643303,72.444582,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000


In [23]:
np.sum(SiteTeam.T.values * expected_rewards) <= 108205.27666406451

True

In [37]:
augmented_graph = pd.DataFrame(expected_rewards)

In [38]:
augmented_graph

,0,1,2,3,4,5,6,7,8,9,...,934,935,936,937,938,939,940,941,942,943
0,136.074522,105.682739,93.000945,258.740714,103.621028,263.242359,302.823868,140.643303,72.444582,225.701586,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
2,136.074522,105.682739,93.000945,0.000000,0.000000,0.000000,302.823868,140.643303,72.444582,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000


In [39]:
extra_columns = pd.DataFrame(np.zeros((19, 19)))

In [40]:
augmented_graph = pd.concat([extra_columns, augmented_graph], axis=1)

In [45]:
augmented_graph.columns = [i for i in range(len(augmented_graph.columns))]

In [46]:
augmented_graph

,0,1,2,3,4,5,6,7,8,9,...,953,954,955,956,957,958,959,960,961,962
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000


In [47]:
temp = expected_rewards.T

In [55]:
temp = pd.DataFrame(temp)

In [56]:
extra_columns = pd.DataFrame(np.zeros((944, 944)))

In [57]:
temp = pd.concat([temp, extra_columns], axis=1)

In [59]:
temp.columns = [i for i in range(len(temp.columns))]

In [60]:
augmented_graph = pd.concat([augmented_graph, temp], axis=0)

In [63]:
import numpy as np
count = np.count_nonzero(augmented_graph.values)
print(count)


3274


In [73]:
test = augmented_graph.values
test[test != 0] = 1
test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [74]:
def is_bipartite(adj_matrix):
    n = len(adj_matrix)
    colors = [0] * n  # Initialize colors for all vertices (0 means uncolored)

    # Assign the first vertex color 1
    colors[0] = 1

    queue = [0]

    while queue:
        u = queue.pop(0)
        for v in range(n):
            if adj_matrix[u][v] == 1:  # Check if v is adjacent to u
                if colors[v] == 0:  # If v is uncolored
                    colors[v] = 3 - colors[u]  # Assign the opposite color of u to v
                    queue.append(v)
                elif colors[v] == colors[u]:  # If v and u have the same color
                    return False  # Graph is not bipartite

    return True  # If no conflicts found, graph is bipartite

# Example usage:
adj_matrix = test
print(is_bipartite(adj_matrix))  # Output: True (Graph is bipartite)


True


In [75]:
def find_cycles(adj_matrix, k):
    n = len(adj_matrix)
    cycles = []

    # Helper function for DFS
    def dfs(node, start, path, length):
        if length > 2 * k:  # If the length exceeds 2k, stop searching
            return
        if length > k and start == node:  # Found a cycle of length at most 2k
            cycles.append(path[:])
            return

        for neighbor in range(n):
            if adj_matrix[node][neighbor] == 1:  # If there's an edge
                if neighbor not in path:  # If the neighbor is not visited
                    dfs(neighbor, start, path + [neighbor], length + 1)

    # Iterate through all nodes and start DFS from each node
    for i in range(n):
        dfs(i, i, [i], 1)

    return cycles

# Example usage:
adj_matrix = [
    [0, 1, 0, 1],
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [1, 0, 1, 0]
]
k = 2
print(find_cycles(adj_matrix, k))  # Output: [[0, 1, 0], [0, 3, 0], [1, 2, 1], [1, 3, 1], [2, 1, 2], [2, 3, 2], [3, 0, 3], [3, 2, 3]]


[]
